In [ ]:
!pip install peft trl datasets

In [ ]:
import torch
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer
from datasets import load_dataset
from transformers import Conv1D, TrainingArguments

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/incoder-1B")
model = AutoModelForCausalLM.from_pretrained("facebook/incoder-1B")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
bool(tokenizer.pad_token is None)

False

In [ ]:
def get_specific_layer_names(model):
    # Create a list to store the layer names
    layer_names = []

    # Recursively visit all modules and submodules
    for name, module in model.named_modules():
        # Check if the module is an instance of the specified layers
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding, torch.nn.Conv2d, Conv1D)):
            # model name parsing

            layer_names.append('.'.join(name.split('.')[4:]).split('.')[0])

    return layer_names

list(set(get_specific_layer_names(model)))

['', 'out_proj', 'v_proj', 'k_proj', 'q_proj']

In [ ]:
lora_config = LoraConfig(
    init_lora_weights="pissa_niter_4",
    r=8,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj'],  # Add target modules here
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
    )
peft_model = get_peft_model(model, lora_config)

peft_model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,638,400 || trainable%: 0.1197


In [ ]:
print(model)

In [ ]:
dataset = load_dataset("Vezora/Tested-143k-Python-Alpaca")

Generating train split:   0%|          | 0/143327 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset['train'],
    dataset_text_field="instruction",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=TrainingArguments(output_dir="./output", num_train_epochs=1)
)
trainer.train()
peft_model.save_pretrained("fine-tunned-model_python_code")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Step,Training Loss
500,10.387700
1000,9.364600
1500,9.217900
2000,9.158100
2500,9.109300
3000,9.072200
3500,9.052100
4000,9.051400
4500,9.018600
5000,9.017600


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

Step,Training Loss
500,10.387700
1000,9.364600
1500,9.217900
2000,9.158100
2500,9.109300
3000,9.072200
3500,9.052100
4000,9.051400
4500,9.018600
5000,9.017600


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/incoder-1B", ) # vocab_size=len(tokenizer)
peft_model = PeftModel.from_pretrained(model, "/content/fine-tunned-model_python_code", ignore_mismatched_sizes=True)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:416: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([50519, 2048]) in the checkpoint and torch.Size([50518, 2048]) in the model instantiated
- base_model.model.model.embed_tokens.weight: found shape torch.Size([50519, 2048]) in the checkpoint and torch.Size([50518, 2048]) in the model instantiated.
  warnings.warn(msg)


In [ ]:
print(peft_model)

## Testing

In [ ]:
import torch

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move the model to the GPU
peft_model.to(device)

input_ids = tokenizer("Defime me a function which can take 2 integers and return the addition of both the numbers.", return_tensors="pt")["input_ids"]

# Move the input tensors to the GPU
input_ids = input_ids.to(device)

# Generate text (now on GPU)
with torch.no_grad():  # Disable gradient calculation for inference
    output = peft_model.generate(input_ids=input_ids, max_new_tokens=500)

# Move the output back to CPU for decoding (if needed)
output = output.cpu()

# Decode the generated text
print(tokenizer.decode(output[0], skip_special_tokens=True))

Using device: cuda
Defime me a function which can take 2 integers and return the addition of both the numbers.
<code>
def add(a, b):
    return a + b
</code>
<|/ a tags=python,python-3.x |>
<| c |>
This doesn't answer the question...
<|/ c |>
<| c |>
This doesn't answer the question...
<|/ c |>
This doesn't answer the question....
<|/ c |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a tags=python,python-3.x |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a |>
<|/ a |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a tags=python,python-3.x |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a tags=python,python-3.x |>
<|/ a |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a and b
<|/ a and b
<|/ a |>
<| a dscore=0 |>
<code>
def add(a, b):
    return a + b
</code>
<|/ a |>
</code>
<|/ a |>
<|/ a |>
<|/ a tags